<a href="https://colab.research.google.com/github/claireluo66/birds_dataset/blob/main/CUBbirds_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.io import read_image, ImageReadMode

from operator import itemgetter
from itertools import groupby
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
!gdown --id 10X4KdsEsqhuZBID-iMAaZMmK6BYZXIVI
!tar -xf CUB_200_2011.tgz

Downloading...
From: https://drive.google.com/uc?id=10X4KdsEsqhuZBID-iMAaZMmK6BYZXIVI
To: /content/CUB_200_2011.tgz
1.15GB [00:05, 213MB/s]


In [5]:
from google.colab import files
uploaded = files.upload()

Saving train_test_val_split.txt to train_test_val_split (1).txt


In [6]:
def parse_datatype(item):
    for datatype in (int, float):
        try:
            return datatype(item)
        except ValueError:
            pass
    return item

def parse_and_filter(f, ids_to_load=None):
    for line in f:
        x = line.split()
        if ids_to_load is None or int(x[0]) in ids_to_load:
            yield [parse_datatype(i) for i in x]

In [8]:
split = [70, 15, 15]

class CUB2011(Dataset):
    def __init__(
        self,
        dataset_path,
        split_file_path="train_test_val_split.txt",
        *,
        type=0,
        images=True,
        labels=True,
        bounding_boxes=False,
        attributes=True,
        part_locs=False,
        mturk_part_locs=False,
        transform=None,
        show_progress=True,
    ):
        root = Path(dataset_path)
        self.transform = transform

        with open(split_file_path) as f:
            self.ids = [id for id, type_ in parse_and_filter(f) if type_ == type]
            ids = set(self.ids)

        if show_progress:
            t = lambda x, desc: tqdm(x, total=len(ids), desc=desc)
        else:
            t = lambda x, _: x

        if images:
            with open(root / "images.txt") as f:
                self.img_paths = {
                    id: root / "images" / fname
                    for id, fname in t(parse_and_filter(f, ids), "images")
                }

        if labels:
            with open(root / "image_class_labels.txt") as f:
                self.labels = {id: label - 1 for id, label in t(parse_and_filter(f, ids), "labels")}

        if bounding_boxes:
            with open(root / "bounding_boxes.txt") as f:
                self.bounding_boxes = {
                    id: torch.tensor(box)
                    for id, *box in t(parse_and_filter(f, ids), "bounding_boxes")
                }

        if attributes:

            def fix_issue(lines):
                for line in lines:
                    if len(line) > 5:
                        yield line[:4] + line[5:]
                    else:
                        yield line

            with open(root / "attributes" / "image_attribute_labels.txt") as f:
                self.attributes = {
                    id: torch.tensor([float(present) for _, _, present, _, _ in fix_issue(lines)])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "attributes"
                    )
                }

        if part_locs:
            with open(root / "parts" / "part_locs.txt") as f:
                self.part_locs = {
                    id: torch.tensor([info for _, *info in lines])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "part_locs"
                    )
                }

        if mturk_part_locs:
            with open(root / "parts" / "part_click_locs.txt") as f:
                self.mturk_part_locs = {
                    id: torch.tensor([info for _, *info, _ in lines])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "mturk_part_locs"
                    )
                }
        
        #self.noise_ratio = noise_ratio
        #if noise_mode:
            #noise_labels = copy.deepcopy(self.labels)
            #idx = list(self.labels.keys()) #11788 images
            #random.shuffle(idx)
            #num_noise = int(self.noise_ratio * len(idx))
            #for i in idx[:num_noise]:
                #noisy_label = random.randint(0,199) #200 classes
                #noise_labels[i] = noisy_label
                
            #print("save noisy labels to %s..."%noise_file)
            #json.dump(noise_labels,open(noise_file,"w"))
        #self.noise_labels = noise_labels

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        id = self.ids[idx]
        item = {}

        if hasattr(self, "img_paths"):
            item["img"] = read_image(str(self.img_paths[id]), mode=ImageReadMode.RGB) / 255
            if self.transform:
                item["img"] = self.transform(item["img"])

        if hasattr(self, "labels"):
            item["label"] = self.labels[id]
        if hasattr(self, "bounding_boxes"):
            item["bounding_box"] = self.bounding_boxes[id]
        if hasattr(self, "attributes"):
            item["attributes"] = self.attributes[id]
        if hasattr(self, "part_locs"):
            item["part_locs"] = self.part_locs[id]
        if hasattr(self, "mturk_part_locs"):
            item["mturk_part_locs"] = self.mturk_part_locs[id]

        return item

In [9]:
preprocess = {
    'train': transforms.Compose([
        transforms.Resize(256), 
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [10]:
train_dataset = CUB2011("CUB_200_2011", type=0, transform=preprocess['train'])
test_dataset = CUB2011("CUB_200_2011", type=1, transform=preprocess['test'])
val_dataset = CUB2011("CUB_200_2011", type=2, transform=preprocess['test'])

In [17]:
class SSLMultimodalModel(nn.Module):
    def __init__(self):
        super().__init__()

        #IMAGES
        
        self.image_model = models.resnet50()
        input_size = self.image_model.fc.out_features

        self.attribute_prediction = [nn.Linear(input_size, 2) for _ in range(312)]
        self.attribute_prediction = nn.ModuleList(self.attribute_prediction)

        #ATTRIBUTES
        attr_model_out_features = 500
        self.attr_model = nn.Sequential(
            nn.Linear(312, 250),
            nn.BatchNorm1d(250),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(250, attr_model_out_features),
            nn.BatchNorm1d(attr_model_out_features),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        fusion_input_size = self.image_model.fc.out_features + attr_model_out_features
        self.rest = nn.Sequential(
            nn.Linear(fusion_input_size, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(500, 200),
        )

    def pretrain(self, imgs):

        image_out = self.image_model(imgs)
        attribute_pred = []
        for i in range(312):
            o = self.attribute_prediction[i](image_out)
            attribute_pred.append(o)
        return attribute_pred

    def forward(self, imgs, attrs):

        image_out = self.image_model(imgs)
        attr_out = self.attr_model(attrs)

        combined = torch.cat([image_out, attr_out], dim=1)
        return self.rest(combined)

In [18]:
model = SSLMultimodalModel()

In [19]:
def default_device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    if isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    return data.to(device)

class DeviceDataLoader():
    def __init__(self, dl, *, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

device = default_device()
print(f"Using Device: {device}")

model.to(device)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
train_dataloader = DeviceDataLoader(train_dataloader, device=device)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
test_dataloader = DeviceDataLoader(test_dataloader, device=device)

val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
val_dataloader = DeviceDataLoader(val_dataloader, device=device)

Using Device: cuda:0


In [20]:
!pip install pkbar

In [21]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [22]:
from pkbar import Kbar

EPOCHS = 5

for epoch in range(EPOCHS):
    kbar = Kbar(target=len(train_dataloader) + len(val_dataloader), epoch=epoch, num_epochs=EPOCHS)

    # Train model

    model.train()
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model.pretrain(batch["img"])
        
        loss = 0
        for j in range(312):
            l = criterion(outputs[j], batch["attributes"][:, j].long()) 
            loss += l
        loss = loss / 312

        loss.backward()
        optimizer.step()

        acc = 0
        for j in range(312):
            pred = outputs[j].argmax(dim=1)
            a = torch.sum(pred == batch["attributes"][:, j].long()) / len(pred)
            acc += a
        acc = acc / 312

        kbar.update(i, values=[("loss", loss.item()), ("acc", acc)])

    # Run validation

    model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(val_dataloader, start=len(train_dataloader)):
            outputs = model.pretrain(batch["img"])

            loss = 0
            for j in range(312):
                l = criterion(outputs[j], batch["attributes"][:, j].long()) 
                loss += l

            acc = 0
            for j in range(312):
                pred = outputs[j].argmax(dim=1)
                a = torch.sum(pred == batch["attributes"][:, j].long()) / len(pred)
                acc += a
            acc = acc / 312

            kbar.update(i, values=[("val_loss", loss.item()), ("val_acc", acc)])
    
    kbar.add(1)

    
print("Done")

Epoch: 1/5
157/157 [==============================] - 139s 887ms/step - loss: 0.2672 - acc: 0.9004 - val_loss: 79.2906 - val_acc: 0.9027
Epoch: 2/5
157/157 [==============================] - 139s 882ms/step - loss: 0.2529 - acc: 0.9029 - val_loss: 79.2597 - val_acc: 0.9024
Epoch: 3/5
157/157 [==============================] - 138s 881ms/step - loss: 0.2493 - acc: 0.9035 - val_loss: 76.5192 - val_acc: 0.9041
Epoch: 4/5
157/157 [==============================] - 138s 881ms/step - loss: 0.2456 - acc: 0.9042 - val_loss: 75.8798 - val_acc: 0.9045
Epoch: 5/5
157/157 [==============================] - 138s 881ms/step - loss: 0.2434 - acc: 0.9048 - val_loss: 74.6822 - val_acc: 0.9054
Done


In [23]:
from pkbar import Kbar

EPOCHS = 20

for epoch in range(EPOCHS):
    kbar = Kbar(target=len(train_dataloader) + len(val_dataloader), epoch=epoch, num_epochs=EPOCHS)

    # Train model

    model.train()
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        outputs = model(batch["img"], batch["attributes"])
        loss = criterion(outputs, batch["label"])
        loss.backward()
        optimizer.step()

        preds = F.softmax(outputs, dim=1).argmax(dim=1)
        acc = torch.sum(batch["label"] == preds) / len(preds)

        kbar.update(i, values=[("loss", loss.item()), ("acc", acc)])

    # Run validation

    model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(val_dataloader, start=len(train_dataloader)):
            outputs = model(batch["img"], batch["attributes"])
            loss = criterion(outputs, batch["label"])

            preds = F.softmax(outputs, dim=1).argmax(dim=1)
            acc = torch.sum(batch["label"] == preds) / len(preds)

            kbar.update(i, values=[("val_loss", loss.item()), ("val_acc", acc)])
    
    kbar.add(1)
    
print("Done")

Epoch: 1/20
157/157 [==============================] - 131s 833ms/step - loss: 5.1136 - acc: 0.0190 - val_loss: 4.8459 - val_acc: 0.0291
Epoch: 2/20
157/157 [==============================] - 130s 828ms/step - loss: 4.8070 - acc: 0.0316 - val_loss: 4.6034 - val_acc: 0.0556
Epoch: 3/20
157/157 [==============================] - 130s 831ms/step - loss: 4.4754 - acc: 0.0777 - val_loss: 4.0895 - val_acc: 0.1447
Epoch: 4/20
157/157 [==============================] - 131s 833ms/step - loss: 4.0418 - acc: 0.1614 - val_loss: 3.6812 - val_acc: 0.2531
Epoch: 5/20
157/157 [==============================] - 131s 836ms/step - loss: 3.6895 - acc: 0.2311 - val_loss: 3.3864 - val_acc: 0.2945
Epoch: 6/20
157/157 [==============================] - 132s 841ms/step - loss: 3.3981 - acc: 0.2997 - val_loss: 3.1396 - val_acc: 0.3648
Epoch: 7/20
157/157 [==============================] - 133s 847ms/step - loss: 3.1652 - acc: 0.3451 - val_loss: 2.9046 - val_acc: 0.4099
Epoch: 8/20
157/157 [====================

In [24]:
def evaluate_model(dataloader=test_dataloader):
    correct = 0
    total = 0

    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            outputs = model(batch["img"], batch["attributes"])
            preds = F.softmax(outputs, dim=1).argmax(dim=1)
            correct += torch.sum(batch["label"] == preds).item()
            total += len(preds)

    print(f"Test Accuracy: {correct}/{total} = {correct/total:.2%}")

In [25]:
evaluate_model()

Test Accuracy: 1028/1773 = 57.98%
